In [ ]:
import os
import json
import time
import azureml
import logging
from azureml.train.dnn import PyTorch
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core import Workspace, Run, Datastore, Experiment

print("Azure ML SDK Version: ", azureml.core.VERSION)

In [ ]:
# use this code to set up config file
#subscription_id ='<SUB_ID>'
#resource_group ='<RESOURCE_GROUP>'
#workspace_name = '<WORKSPACE>'

#try:
#    ws = Workspace(subscription_id = subscription_id, resource_group = resource_group, workspace_name = workspace_name)
#    ws.write_config()
#    print('Workspace configuration succeeded. You are all set!')
#except:
#    print('Workspace not found. TOO MANY ISSUES!!!')

ws = Workspace.from_config()

# Compute Environment

In [ ]:
cluster = 'sauron'
try:
    compute = ComputeTarget(workspace=ws, name=cluster)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', min_nodes=1, max_nodes=6)
    compute = ComputeTarget.create(ws, cluster, compute_config)
    compute.wait_for_completion(show_output=True)

# Manage Data

In [ ]:
# Register datastore
#ds = Datastore.register_azure_blob_container(workspace=ws,
#                                             datastore_name='foodai',
#                                             container_name='foodai',
#                                             account_name='robotdatasets',
#                                             account_key='<KEY>',
#                                             create_if_not_exists=False)
for name, ds in ws.datastores.items():
    print(name, ds.datastore_type)

In [ ]:
ds = Datastore.get(ws, datastore_name='foodai')

# Run Experiment

In [ ]:
script_params = {
    "--data": ds.as_mount(), # don't download
    "--output": "outputs",
    "--epochs": 25,
    "--batch": 8,
    "--lr": 0.001,
}

# Create and run experiment
foodEstimator = PyTorch(source_directory='./',
                        script_params=script_params,
                        compute_target=compute,
                        entry_script='train.py',
                        conda_packages=['matplotlib'],
                        use_gpu=True)

In [ ]:
exp = Experiment(ws, 'food_ai')
run = exp.submit(foodEstimator)
run

In [ ]:
RunDetails(run).show()

# Save Model

In [ ]:
from azureml.core.model import Model
run.get_file_names()

In [ ]:
run.download_file(name='outputs/model.onnx', output_file_path='outputs')
model_file = 'outputs/model.onnx'
model = Model.register(ws, model_name='FoodAI', model_path=model_file, 
                       description='AI to detect Hot Dog vs Pizza')

# Create Image

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies
myenv = CondaDependencies()

myenv.add_pip_package('numpy')
myenv.add_pip_package('pillow')
myenv.add_pip_package('requests')
myenv.add_pip_package('torchvision')
myenv.add_pip_package('onnxruntime')

with open('foodai_scoring.yml','w') as f:
    print('Writing out {}'.format('foodai_scoring.yml'))
    f.write(myenv.serialize_to_string())
    print('Done!')

In [ ]:
from azureml.core.image import ContainerImage, Image
image_config = ContainerImage.image_configuration(execution_script="score.py", 
                                    runtime="python", 
                                    conda_file="foodai_scoring.yml")


image = Image.create(ws, 'foodai', [model], image_config)
image.wait_for_creation()

In [ ]:
image.image_build_log_uri

# Deploy Image

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.webservice import Webservice

service_name = 'foodai-svc'
if service_name in ws.webservices:
    ws.webservices[service_name].delete()

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1, 
                                               memory_gb=1, 
                                               description='FoodAI Service')
service = Webservice.deploy_from_image(workspace=ws, 
                                       image=image, 
                                       name=service_name, 
                                       deployment_config=aciconfig)
service.wait_for_deployment(show_output=True)

In [ ]:
with open('deploy.log','w') as f:
    f.write(service.get_logs())
service.scoring_uri